In [7]:
# 실제 train data를 변환하기 - 데이터 읽어들이기

import json
import os
import pandas as pd
import numpy as np

base_dir = os.getcwd()

train_images_path = base_dir + '/data/train/images/'
train_labels_json_path = base_dir + '/data/train/labels_json/'
train_labels_path = base_dir + '/data/train/labels/'

os.makedirs(train_images_path, exist_ok=True)
os.makedirs(train_labels_json_path, exist_ok=True)
os.makedirs(train_labels_path, exist_ok=True)

!unzip -qq '/content/drive/MyDrive/KCC3기_2차_프로젝트/train.zip' -d '/content/data/train'

train_images_list = os.listdir(train_images_path)
train_labels_json_list = os.listdir(train_labels_json_path)

train_images_list.sort()
train_labels_json_list.sort()

In [15]:
# 실제 train data를 변환하기 - 라벨링 변환

yolo_class = {
    '철캔': 0,
    '알루미늄캔': 1,
    '종이': 2,
    '무색단일': 3,
    '유색단일': 4,
    'PE': 5,
    'PP': 6,
    'PS': 7,
    '스티로폼': 8,
    '비닐': 9,
    '갈색': 10,
    '녹색': 11,
    '투명': 12
}

for label in train_labels_json_list:
  file_json = open(train_labels_json_path + label)
  f_json = json.load(file_json)

  img_width = f_json['IMAGE_INFO']['IMAGE_WIDTH']
  img_height = f_json['IMAGE_INFO']['IMAGE_HEIGHT']

  # print(img_width)
  # print(img_height)

  f_txt = open(train_labels_path + label.replace('.json', '.txt'), 'w')

  for anno in f_json['ANNOTATION_INFO']:
    obj_id = yolo_class[anno['DETAILS']]
    obj_points = np.array(anno['POINTS'])
    # obj_points = np.array(anno['POINTS'][0])

    # print(obj_id)
    # print(obj_points)
    if obj_points[0].size == 4:
      yolo_id = obj_id
      yolo_points = np.array([obj_points[0][0]+obj_points[0][2]/2, obj_points[0][1]+obj_points[0][3]/2,
                              obj_points[0][2], obj_points[0][3]])

      yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                             yolo_points[2]/img_width, yolo_points[3]/img_height])

      # print(yolo_points)
      # print(yolo_points_norm)

      yolo_points_norm = list(yolo_points_norm)
      f_txt.write(str(yolo_id) + ' ')
      f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
      f_txt.write('\n')
    elif obj_points.size >= 2:
      max_x = 0
      min_x = img_width
      max_y = 0
      min_y = img_height
      for points in obj_points:
        if points[0] > max_x:
          max_x = points[0]
        if points[0] < min_x:
          min_x = points[0]
        if points[1] > max_y:
          max_y = points[1]
        if points[1] < min_y:
          min_y = points[1]

      yolo_id = obj_id
      yolo_points = np.array([(min_x+max_x)/2, (min_y+max_y)/2,
                              max_x-min_x, max_y-min_y])
      yolo_points_norm = np.array([yolo_points[0]/img_width, yolo_points[1]/img_height,
                             yolo_points[2]/img_width, yolo_points[3]/img_height])
      yolo_points_norm = list(yolo_points_norm)

      f_txt.write(str(yolo_id) + ' ')
      f_txt.write(' '.join(str(ele) for ele in yolo_points_norm))
      f_txt.write('\n')
    else:
      print(label)
  f_txt.close()


In [17]:
os.listdir('/content/data/train/labels_json')[:5]

print(open('/content/data/train/labels/606511@4_04002_220907_P1_T1.txt').readline())
print(open('/content/data/train/labels/10349@3_01001_220715_P1_T1.txt').readline())


6 0.4915269230769231 0.49103166345617255 0.38315604395604386 0.6721641025641029

0 0.6400563522733375 0.3877540579713878 0.1944409271364939 0.387188312642922



In [6]:
# import shutil

# shutil.rmtree('/content/data')